<a href="https://colab.research.google.com/github/Katou28/DISC_Personality_Test/blob/main/PhisNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is i

In [2]:
from datasets import load_dataset
dataset = load_dataset("zefang-liu/phishing-email-dataset")
print(dataset)
print(dataset['train'][0]['Email Type'])
print(dataset['train'].features['Email Type'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/616 [00:00<?, ?B/s]

Phishing_Email.csv:   0%|          | 0.00/52.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18650 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Email Text', 'Email Type'],
        num_rows: 18650
    })
})
Safe Email
Value(dtype='string', id=None)


In [3]:
#splitting the dataset for test/train
from datasets import ClassLabel, DatasetDict
import pandas as pd

print("Casting 'Email Type' column to ClassLabel...")
# Get the unique string labels from the column
unique_labels = dataset['train'].unique('Email Type')
# Create the ClassLabel feature using these unique labels
class_label_feature = ClassLabel(names=unique_labels)
# Apply the casting to the 'train' split
dataset['train'] = dataset['train'].cast_column('Email Type', class_label_feature)

# Verify the change (optional but recommended)
print("\n--- Feature after casting 'Email Type' ---")
print(dataset['train'].features)
# Check the integer label for the first example
print("\nExample label after casting (now an integer):")
print(dataset['train'][0]['Email Type'])


# --- Now, re-run the splitting code ---
print("\nSplitting the dataset with stratification...")

# 1. Split the original 'train' data into training (80%) and test (20%)
# Stratify by 'Email Type' should work now because it's a ClassLabel
train_test_split = dataset['train'].train_test_split(test_size=0.2, stratify_by_column='Email Type')

test_dataset = train_test_split['test']         # This is 20% of the original data

# 2. Split the intermediate training set (80% of original) into final train and validation
# Validation = 10% of original => 12.5% of this intermediate training set (0.1 / 0.8 = 0.125)
train_val_split = train_test_split['train'].train_test_split(test_size=0.125, stratify_by_column='Email Type')

train_dataset = train_val_split['train']         # This is the final training set (70% of original)
validation_dataset = train_val_split['test']     # This is the final validation set (10% of original)

# 3. Combine them back into a single DatasetDict
final_dataset = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset
})

# Print the final dataset structure to verify
print("\n--- Final Dataset Structure ---")
print(final_dataset)

# Optional: Check the class distribution in each split
print("\n--- Class Distribution Check ---")
for split_name, split_data in final_dataset.items():
    print(f"\nSplit: {split_name}")
    try:
        df = split_data.to_pandas()
        # Labels are now integers (0, 1, etc.)
        print("Label distribution (normalized):")
        print(df['Email Type'].value_counts(normalize=True))
        print("\nLabel distribution (raw counts):")
        print(df['Email Type'].value_counts())
    except ImportError:
        print("Pandas library not found. Skipping distribution check.")
    except Exception as e:
        print(f"Could not check distribution for {split_name}: {e}")

Casting 'Email Type' column to ClassLabel...


Casting the dataset:   0%|          | 0/18650 [00:00<?, ? examples/s]


--- Feature after casting 'Email Type' ---
{'Unnamed: 0': Value(dtype='int64', id=None), 'Email Text': Value(dtype='string', id=None), 'Email Type': ClassLabel(names=['Safe Email', 'Phishing Email'], id=None)}

Example label after casting (now an integer):
0

Splitting the dataset with stratification...

--- Final Dataset Structure ---
DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Email Text', 'Email Type'],
        num_rows: 13055
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'Email Text', 'Email Type'],
        num_rows: 1865
    })
    test: Dataset({
        features: ['Unnamed: 0', 'Email Text', 'Email Type'],
        num_rows: 3730
    })
})

--- Class Distribution Check ---

Split: train
Label distribution (normalized):
Email Type
0    0.607124
1    0.392876
Name: proportion, dtype: float64

Label distribution (raw counts):
Email Type
0    7926
1    5129
Name: count, dtype: int64

Split: validation
Label distribution (normalized):
Emai

In [4]:
# Tokenizing - crucial step
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-uncased"

# 2. Load the tokenizer associated with the chosen checkpoint
print(f"Loading tokenizer for '{model_checkpoint}'...")
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
print("Tokenizer loaded.")

# 3. Define a function to tokenize the email text
#    This function takes a batch of examples and applies the tokenizer to the 'Email Text' field.
def tokenize_function(examples):
    # padding='max_length' pads shorter sequences to the model's max length.
    # truncation=True cuts off sequences longer than the model's max length.
    # Convert 'Email Text' values to a list of strings if necessary
    email_texts = examples["Email Text"]
    # Ensure email_texts is always a list of strings
    email_texts = [str(text) if not isinstance(text, str) else text for text in email_texts]
    return tokenizer(email_texts, padding="max_length", truncation=True)

# 4. Apply the tokenization function to all splits in your dataset
#    We use batched=True for faster processing.
print("Applying tokenizer to the dataset (train, validation, test)...")
# This assumes your dataset dict from the previous step is named 'final_dataset'
tokenized_datasets = final_dataset.map(tokenize_function, batched=True)
print("Tokenization complete.")

# 5. Inspect results
print("\n--- Structure after tokenization ---")
print(tokenized_datasets)
print("\n--- Example of tokenized features (first train example) ---")
print(tokenized_datasets["train"][0])

Loading tokenizer for 'distilbert-base-uncased'...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Tokenizer loaded.
Applying tokenizer to the dataset (train, validation, test)...


Map:   0%|          | 0/13055 [00:00<?, ? examples/s]

Map:   0%|          | 0/1865 [00:00<?, ? examples/s]

Map:   0%|          | 0/3730 [00:00<?, ? examples/s]

Tokenization complete.

--- Structure after tokenization ---
DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Email Text', 'Email Type', 'input_ids', 'attention_mask'],
        num_rows: 13055
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'Email Text', 'Email Type', 'input_ids', 'attention_mask'],
        num_rows: 1865
    })
    test: Dataset({
        features: ['Unnamed: 0', 'Email Text', 'Email Type', 'input_ids', 'attention_mask'],
        num_rows: 3730
    })
})

--- Example of tokenized features (first train example) ---
{'Unnamed: 0': 17180, 'Email Text': 'hi again , here is don duran . i wite you because we are accepting your mortgage application . our office confirms you can get a $ 220 . 000 lo?n for a $ 352 . 00 per month payment . approval process will take 1 minute , so please fill out the form on our website : thank you . best regards don duran first account manager', 'Email Type': 1, 'input_ids': [101, 7632, 2153, 1010, 2182, 20

In [5]:
#training - original
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from transformers import AutoTokenizer # Import AutoTokenizer

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# 1. Data Collation
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# 2. Model Loading and Setup
num_labels = len(final_dataset['train'].unique('Email Type'))  # Get number of labels
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

# 3. Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1, # 1 training pass, (you can have more passes but takes longer training)
    weight_decay=0.01,
    evaluation_strategy="epoch",
)

#Rename the 'Email Type' column to 'labels'
tokenized_datasets = tokenized_datasets.rename_column("Email Type", "labels")

# 4. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    # Add metrics if needed
)

# 5. Training and Evaluation
trainer.train()
eval_results = trainer.evaluate()
print(eval_results)

# 6. Save the Model (Optional)
trainer.save_model("./my_phishing_model")

Flattening the indices:   0%|          | 0/13055 [00:00<?, ? examples/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: melindocharls (melindocharls-university-of-science-and-technology-of-so) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.147200,0.070033


{'eval_loss': 0.07003255933523178, 'eval_runtime': 1174.4599, 'eval_samples_per_second': 1.588, 'eval_steps_per_second': 0.1, 'epoch': 1.0}
